In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

## Lets get the data, model and setup training code

In [26]:
train_loader = DataLoader(datasets.MNIST("./", train=True, transform=transforms.ToTensor(), download=True), batch_size=128, shuffle=True)
test_loader = DataLoader(datasets.MNIST("./", train=False, transform=transforms.ToTensor(), download=True), batch_size=128, shuffle=False)

print(f"Training images {len(train_loader.dataset)}, Test images {len(test_loader.dataset)}")

Training images 60000, Test images 10000


In [27]:
class mnist_model(nn.Module):
  def __init__(self):
    super(mnist_model, self).__init__()
    self.layer1 = nn.Linear(784, 128, bias=True)
    self.layer2 = nn.Linear(128, 128, bias=True)
    self.layer3 = nn.Linear(128, 10, bias=True)
    # self.act = nn.Hardtanh()
    self.act = nn.ReLU()

  def forward(self, x):
    return self.layer3(self.act(self.layer2(self.act(self.layer1(x)))))

  def output(self, x):
    out1 = self.act(self.layer1(x))
    out2 = self.act(self.layer2(out1))
    out3 = self.layer3(out2)
    return out1, out2, out3

In [28]:
model = mnist_model()
print(model)

epochs = 15
lr = 0.1

USE_MSE = True
optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = nn.MSELoss() if USE_MSE else nn.CrossEntropyLoss()
# lrs = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

mnist_model(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=10, bias=True)
  (act): ReLU()
)


## Training

In [29]:
def get_acc(model, loader):
  correct = 0
  total = 0
  for img, label in loader:
    correct += torch.sum(torch.argmax(model(img.view(-1, 784)), -1) == label).item()
    total += len(img)
  return 100*correct/total

In [30]:
for e in range(epochs):
  print("lr", optimizer.param_groups[0]["lr"])
  for img, label in train_loader:
    out = model(img.view(-1, 784))
    label_onehot = nn.functional.one_hot(label, num_classes=10).float()
    optimizer.zero_grad()
    loss = criterion(out, label_onehot)
    loss.backward()
    optimizer.step()
  # lrs.step()
  print(f"Epoch {e}, training accuracy {get_acc(model, train_loader)}, test accuracy {get_acc(model, test_loader)}")

lr 0.1
Epoch 0, training accuracy 81.31166666666667, test accuracy 82.47
lr 0.1
Epoch 1, training accuracy 87.10333333333334, test accuracy 87.75
lr 0.1
Epoch 2, training accuracy 89.18833333333333, test accuracy 89.95
lr 0.1
Epoch 3, training accuracy 90.31333333333333, test accuracy 90.96
lr 0.1
Epoch 4, training accuracy 91.17, test accuracy 91.78
lr 0.1
Epoch 5, training accuracy 91.83, test accuracy 92.3
lr 0.1
Epoch 6, training accuracy 92.25833333333334, test accuracy 92.85
lr 0.1
Epoch 7, training accuracy 92.77833333333334, test accuracy 93.08
lr 0.1
Epoch 8, training accuracy 93.11333333333333, test accuracy 93.37
lr 0.1
Epoch 9, training accuracy 93.36333333333333, test accuracy 93.78
lr 0.1
Epoch 10, training accuracy 93.67833333333333, test accuracy 93.93
lr 0.1
Epoch 11, training accuracy 93.87166666666667, test accuracy 94.14
lr 0.1
Epoch 12, training accuracy 94.05833333333334, test accuracy 94.28
lr 0.1
Epoch 13, training accuracy 94.27, test accuracy 94.57
lr 0.1
Epoc

## Extract weights

In [31]:
params = [(name, p.data.cpu().numpy()) for (name, p) in model.named_parameters()]

In [32]:
for (name, p) in params:
  print(f"Layer {name.split('.')[0]}, type {name.split('.')[1]}, shape {p.shape}")

Layer layer1, type weight, shape (128, 784)
Layer layer1, type bias, shape (128,)
Layer layer2, type weight, shape (128, 128)
Layer layer2, type bias, shape (128,)
Layer layer3, type weight, shape (10, 128)
Layer layer3, type bias, shape (10,)


In [33]:
# print(params)

In [34]:
for img, label in train_loader:
  break

In [35]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/Colab Notebooks/Falcon Neural Network/SecureML/"

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os

np.savetxt(fname=path+"input_0", delimiter=" ", X=img.cuda().view(-1, 784).tolist())
np.savetxt(fname=path+"outputlayer1_0", delimiter=" ", X=model.output(img.cuda().view(-1, 784))[0].tolist())
np.savetxt(fname=path+"outputlayer2_0", delimiter=" ", X=model.output(img.cuda().view(-1, 784))[1].tolist())
np.savetxt(fname=path+"outputlayer3_0", delimiter=" ", X=model.output(img.cuda().view(-1, 784))[2].tolist())

np.savetxt(fname=path+"weight1_0", delimiter=" ", X=params[0][1].tolist())
np.savetxt(fname=path+"bias1_0", delimiter=" ",  X=params[1][1].tolist())
np.savetxt(fname=path+"weight2_0", delimiter=" ", X=params[2][1].tolist())
np.savetxt(fname=path+"bias2_0", delimiter=" ", X=params[3][1].tolist())
np.savetxt(fname=path+"weight3_0", delimiter=" ", X=params[4][1].tolist())
np.savetxt(fname=path+"bias3_0", delimiter=" ", X=params[5][1].tolist())